In [ ]:
import src.h5Interface as h5i
import src.reactorInterface as RI
import os
import pickle

from scipy.interpolate import interp1d as interp
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

import osbrain
from osbrain import run_nameserver
from osbrain import run_agent
from osbrain import logging
import numpy as np
import src.blackboard as blackboard
import src.ka as ka
import src.ka_rp as karp
import src.ka_br as kabr
import src.bb_opt as bb_opt
import src.controller as controller
import time
import src.performance_measure as pm
import math
import h5py

import numpy as np
import pandas as pd


In [ ]:
objectives = {'reactivity swing': {'ll':250,  'ul':1000,  'goal':'lt', 'variable type': float},
              'burnup':           {'ll':30,   'ul':80,    'goal':'gt', 'variable type': float},
              'pu mass':          {'ll':0,    'ul':1000,  'goal':'lt', 'variable type': float}}
#objectives = {'reactivity swing': {'ll':250,   'ul':1000,  'goal':'lt', 'variable type': float},
#              'burnup':           {'ll':30,   'ul':80,    'goal':'gt', 'variable type': float},
#              'pu mass':          {'ll':0,   'ul':1000, 'goal':'lt', 'variable type': float},
#              'cycle length':     {'ll': 120, 'ul': 360, 'goal':'gt', 'variable type': float}}
objectives = {'reactivity swing': {'ll':250, 'ul':1500,  'goal':'lt', 'variable type': float},
              'burnup':           {'ll':20,  'ul':100,    'goal':'gt', 'variable type': float},
              'pu mass':          {'ll':0,    'ul':1000,  'goal':'lt', 'variable type': float},
              'cycle length':     {'ll':60,  'ul':360, 'goal':'gt', 'variable type': float}}

objectives = {'reactivity swing': {'ll':250,   'ul':1000, 'goal':'lt', 'variable type': float},
              'burnup':           {'ll':30,   'ul':80,  'goal':'gt', 'variable type': float}}
design_variables = {'height':     {'ll': 50.0, 'ul': 100.0, 'variable type': float},
                    'smear':      {'ll': 50.0, 'ul': 80.0, 'variable type': float},
                    'pu_content': {'ll': 0.0,  'ul': 1.0,  'variable type': float}}
kas = {'ka_rp_mc': karp.KaGlobal, 
       'ka_rp_hc': karp.KaLocalHC,
#       'ka_rp_hc2': karp.KaLocalHC,
#       'ka_rp_pert': karp.KaLocal,
#       'ka_rp_ga': karp.KaGA,
       'ka_br_lvl3': kabr.KaBr_lvl3,
       'ka_br_lvl2': kabr.KaBr_lvl2,
       'ka_br_lvl1': kabr.KaBr_lvl1}

full_div = {'reactivity swing': 625, 'burnup': 160}#, 'cycle length': 300, 'pu mass': 500}

for x in range(0,1):#range(25,26):
    bb_controller = controller.Controller(bb_name='sfr_opt', 
                                          bb_type=bb_opt.BbOpt, 
                                          ka=kas, 
                                          objectives=objectives,
                                          design_variables=design_variables,
                                          constraints={},
                                          archive='sfr_bb_{}_testing'.format(x), 
                                          agent_wait_time=60*120, 
                                          plot_progress=True,
                                          convergence_model={'type':'dci hvi', 'convergence rate': 1E-6, 'div': full_div},
                                          progress_rate=20,
                                          surrogate_model={'sm_type': 'gpr', 'pickle file': 'sm_gpr_120days.pkl'})
    bb = bb_controller.bb
    bb.set_attr(num_calls=10)
    #bb.set_attr(_DEBUG=True)
    ka = bb.get_attr('_proxy_server')
    ka1 = ka.proxy('ka_rp_mc')
    ka2 = ka.proxy('ka_rp_hc')
#    ka5 = ka.proxy('ka_rp_hc2')
 #   ka3 = ka.proxy('ka_rp_pert')
    br1 = ka.proxy('ka_br_lvl1')
    br1.set_attr(total_pf_size=200)
    br1.set_attr(pareto_sorter='dci')
#    br1.set_attr(dci_div={'reactivity swing': 150, 'burnup': 50, 'pu mass': 100})
    br1.set_attr(dci_div=full_div)
    for ka_ in [ka1, ka2]:#, ka3]:
        ka_.set_attr(_design_accuracy=3)
        ka_.set_attr(_objective_accuracy=3)
    ka1.set_attr(_base_trigger_val=1)
    ka2.set_attr(step_limit=15)
#    ka5.set_attr(hc_type='steepest ascent')
#    ka5.set_attr(step_limit=25)
    
#    ka4 = ka.proxy('ka_rp_ga')    
 #   ka4.set_attr(crossover_type='linear crossover')
    
    bb.set_attr(hv_convergence=1E-6)
    bb_controller.run_single_agent_bb()
    print(bb.get_attr('abstract_lvls')['level 100']['final'])
    print(len(bb.get_attr('abstract_lvls')['level 1']))
    bb_controller.shutdown()
    time.sleep(2)

In [ ]:
ns = run_nameserver()
bb = run_agent(name='bb', base=bb_opt.BbOpt)
bb.set_attr(archive_name='sfr_bb_0_4d.h5')
bb.load_h5(panels={2:['new','old'],3:['new','old']})
lvl2 = []
lvl3 = []
lvl1 = []
mc = []
hc = []
ga = []
pert = []
print(len(bb.get_attr('abstract_lvls')['level 1']))
for trigger in bb.get_attr('abstract_lvls')['level 100'].values():
    if 'lvl3' in trigger['agent']:
        lvl3.append(trigger['time'])
    elif 'lvl2' in trigger['agent']:
        lvl2.append(trigger['time'])
    elif 'lvl1' in trigger['agent']:
        lvl1.append(trigger['time'])
    elif 'mc' in trigger['agent']:
        mc.append(trigger['time'])
    elif 'hc' in trigger['agent']:
        hc.append(trigger['time'])
    elif 'ga' in trigger['agent']:
        ga.append(trigger['time'])
    elif 'pert' in trigger['agent']:
        pert.append(trigger['time'])
        
#dict_ = {'lvl3': }
#df = pd.DataFrame(list(zip(lvl3, lvl2, lvl1)), columns=['time', 'time', 'lvl1'])
#print(df)
#fig1 = px.line(df, y='time',
#                  labels={'x':'times triggered', 'y': 'time'})
#fig1.show()
fig = go.Figure()
for level in [(lvl3, 'level 3'), (lvl2, 'level 2'), (lvl1, 'level 1')]:#, mc, hc, ga, pert]:
    fig.add_trace(go.Scatter(x=[i for i in range(len(level[0]))], y=level[0], name=level[1]))

fig.update_layout(xaxis_title='Times Triggered',
                  yaxis_title='Time (s)')
fig.show()

fig = go.Figure()
for level in [(mc, 'MC'), (hc, 'HC'), (ga, 'GA'), (pert, 'Pert')]:#, mc, hc, ga, pert]:
    fig.add_trace(go.Scatter(x=[i for i in range(len(level[0]))], y=level[0], name=level[1]))

fig.update_layout(xaxis_title='Times Triggered',
                  yaxis_title='Time (s)')
fig.show()

In [ ]:
def mean_std(results):
    mean = sum([x for x in results]) / len(results)
    var  = sum(pow(x-mean,2) for x in results) / len(results)
    std  = math.sqrt(var)
    return mean, std

def max_pf_val(results, objective):
    max_ = -1E10
    for core in results.values():
        max_ = core[objective] if core[objective] > max_ else max_
    return max_

def min_pf_val(results, objective):
    min_ = 1E10
    for core in results.values():
        min_ = core[objective] if core[objective] < min_ else min_
    return min_

def obj_range(min_, max_):
    return (max_ - min_)

def objective_range(results, objective):
    return max_pf_val(results, objective) - min_pf_val(results, objective)

def calculate_hvi(pf, ll, ul):
    hvi = pm.hypervolume_indicator(pf, ll, ul)
    return hvi

def calculate_dci(ll, ul, pfs, goal, div):
    dci = pm.diversity_comparison_indicator(ll, ul, pfs, goal=goal, div=div)
    return dci

def read_ga_pf(ga_file, design_variables, objectives, num):
    ga_dir = '/Users/ryanstewart/projects/Dakota_interface/' + ga_file 
    f = open(ga_dir + 'final_{}.dat'.format(num))
    ga_pf = {}

    for line in f:
        split_line = line.split('\t')
        split_line = [float(x) for x in split_line]
        name = 'core_[{}, {}, {}]'.format(split_line[0], split_line[1], split_line[2])
        ga_pf[name] = {}
#        for n, dv in enumerate(design_variables.keys()):
 #           ga_pf[name].update({dv: split_line[n]})
        for n, obj in enumerate(objectives.keys()):
            ga_pf[name].update({obj: split_line[n+len(design_variables)]})

    f.close()
    f = open(ga_dir + 'sfr_moo_bb_comp_2_{}.out'.format(num, 'r'))
    for line in f:
        if 'wall clock' in line:
            split_line = line.split('  ')
            ga_time = float(split_line[-1])
    f.close()
    
    for core in ga_pf.values():
        for obj in core.keys():
            if obj == 'burnup':
                core[obj] = -core[obj]
                
    pf = [[core[obj] for obj in objectives.keys()] for core in ga_pf.values()]
    ga_hvi = calculate_hvi(pf, [x['ll'] for x in objectives.values()], [x['ul'] for x in objectives.values()])
    for core in ga_pf.values():
        for obj in core.keys():
            if obj == 'burnup':
                core[obj] = -core[obj]

    return ga_pf, ga_time, ga_hvi

def read_bb_pf(bb_file, design_variables, objectives, num):
    ns = run_nameserver()
    bb = run_agent(name='bb', base=bb_opt.BbOpt)
    bb.set_attr(archive_name=bb_file+'sfr_bb_{}.h5'.format(num))
    bb.load_h5(panels={2:['new','old'],3:['new','old']})
    lvls = bb.get_attr('abstract_lvls')
    lvl1 = lvls['level 1']    
    lvl3 = {}
    for name in lvls['level 3'].values():
        lvl3.update(name)
    bb_pf = {}
    for name, rx_parameters in lvl1.items():
        bb_pf[name] = {}
#        for num, dv in enumerate(design_variables.keys()):
 #           bb_pf[name].update({dv: lvl3[name]['design variables'][dv]})
        for num, obj in enumerate(objectives.keys()):
            bb_pf[name].update({obj: lvl3[name]['objective functions'][obj]})
    meta_data = lvls['level 100']['final']
    
    ns.shutdown()
    time.sleep(1)
    return bb_pf, meta_data['time'], meta_data['hvi']
    
def compare_results(num_to_compare, ga_dir, bb_dir, objectives, design_variabes, div, goal):
    ga_pfs = {}
    bb_pfs = {}
    full_pf = []
    for i in range(11,20):
        try:
            ga_pf, ga_time, ga_hvi = read_ga_pf(ga_dir, design_variables, objectives, i)        
            full_pf.append(ga_pf)
            ga_pfs[i] = {'pf': ga_pf, 'time': ga_time, 'hvi': ga_hvi}
            print(ga_pfs[i]['hvi'])
        except:
            print('Failed to load ga {}'.format(i))
        bb_pf, bb_time, bb_hvi = read_bb_pf(bb_dir, design_variables, objectives, i)
        full_pf.append(bb_pf)
        bb_pfs[i] = {'pf': bb_pf, 'time': bb_time, 'hvi': bb_hvi}

    
    ll = {obj_name: obj['ll'] for obj_name, obj in objectives.items()}
    ul = {obj_name: obj['ul'] for obj_name, obj in objectives.items()}

    dci_object = calculate_dci(ll, ul, full_pf, goal, div)
    for i in range(11,20):
        dci_object.compute_dci(ga_pfs[i]['pf'])
        ga_pfs[i].update({'dci': dci_object.dci})
        dci_object.compute_dci(bb_pfs[i]['pf'])
        bb_pfs[i].update({'dci': dci_object.dci})
    
    results = ['dci', 'hvi', 'time',]
    ga_temp  ={}
    for i in range(11,20):
        ga_temp['time'] = ga_pfs[i]['time']
        ga_temp['dci'] = ga_pfs[i]['dci']
        ga_temp['hvi'] = ga_pfs[i]['hvi']
    return ga_pfs, bb_pfs

In [ ]:
objective_3d = {'reactivity swing': {'ll':250,  'ul':1000,  'goal':'lt', 'variable type': float},
             'burnup':           {'ll':-80,   'ul':-30,    'goal':'gt', 'variable type': float},
             'pu mass':          {'ll':0,    'ul':1000,  'goal':'lt', 'variable type': float}}
objective_2d = {'reactivity swing': {'ll':250,  'ul':1000,  'goal':'lt', 'variable type': float},
                'burnup':           {'ll':-80,   'ul':-30,    'goal':'gt', 'variable type': float}}
design_variables = {'height':     {'ll': 50.0, 'ul': 100.0, 'variable type': float},
                    'smear':      {'ll': 50.0, 'ul': 80.0, 'variable type': float},
                    'pu_content': {'ll': 0.0,  'ul': 1.0,  'variable type': float}}
div_2d={'reactivity swing':  150, 'burnup':  100}        
goal_2d={'reactivity swing':  'lt', 'burnup':  'gt'} 
div_3d={'reactivity swing':  150, 'burnup':  100, 'pu mass': 100}        
goal_3d={'reactivity swing':  'lt', 'burnup':  'gt', 'pu mass': 'lt'} 
bb_file_2d = 'mc_2d_1E5/'
ga_file_2d = 'GA_BB_2D/'
bb_file_3d = 'mc_3d_1E5/'
#bb_file_3d = 'mc_3d_1E6/'
ga_file_3d = 'GA_BB_3D/'

time_1 = time.time()
ga, bb = compare_results(9, ga_file_3d, bb_file_3d, objective_3d, design_variables, div_3d, goal_3d)
#ga, bb = compare_results(20, ga_file_2d, bb_file_2d, objective_2d, design_variables, div_2d, goal_2d)
print(time.time()-time_1)
a = 1

In [ ]:
hvi = mean_std([x['hvi'] for x in ga.values()])
dci = mean_std([x['dci'] for x in ga.values()])
time_ = mean_std([x['time'] for x in ga.values()])
s_rx = mean_std([objective_range(x['pf'], 'reactivity swing') for x in ga.values()])
s_bu = mean_std([objective_range(x['pf'], 'burnup') for x in ga.values()])
s_pu = (0,0)#mean_std([objective_range(x['pf'], 'pu mass') for x in ga.values()])
size_ = mean_std([len(x['pf']) for x in ga.values()])
print(size_)

print('GA & {} ({}) & {} ({}) & {} ({}) &  {} ({}) & {} ({}) & {} ({}) \\\ '.format(round(hvi[0], 4), round(hvi[1], 4), 
                                                                     round(dci[0], 3), round(dci[1], 3),
                                                                     int(s_rx[0]), int(s_rx[1]),
                                                                     round(s_bu[0], 1), round(s_bu[1], 1),
                                                                     round(s_pu[0], 1), round(s_pu[1], 1),
                                                                     int(time_[0]), int(time_[1])))
hvi = mean_std([x['hvi'] for x in bb.values()])
dci = mean_std([x['dci'] for x in bb.values()])
time_ = mean_std([x['time'] for x in bb.values()])
s_rx = mean_std([objective_range(x['pf'], 'reactivity swing') for x in bb.values()])
s_bu = mean_std([objective_range(x['pf'], 'burnup') for x in bb.values()])
s_pu = (0,0)#mean_std([objective_range(x['pf'], 'pu mass') for x in bb.values()])
size_ = mean_std([len(x['pf']) for x in bb.values()])
print(size_)
print('ABBS & {} ({}) & {} ({}) & {} ({}) &  {} ({}) & {} ({}) & {} ({}) \\\ '.format(round(hvi[0], 4), round(hvi[1], 4), 
                                                                     round(dci[0], 3), round(dci[1], 3),
                                                                     int(s_rx[0]), int(s_rx[1]),
                                                                     round(s_bu[0], 1), round(s_bu[1], 1),
                                                                     round(s_pu[0], 1), round(s_pu[1], 1),
                                                                     int(time_[0]), int(time_[1])))

In [ ]:
a = [0.643549871792158, 0.6516215944933403, 0.643549871792158, 0.6323760186170366, 0.6453504760269841, 
     0.6568673635625429,0.6493726605513199,0.6587769145626935, 0.6335473138678313, 0.6528636485203724,
     0.6538635755721666, 0.6435927012664061, 0.649511265712514, 0.6462687521193944,0.649511265712514,
    0.6462687521193944, 0.6303246266955884, 0.6360361181757995, 0.6489716459032033, 0.6571378893882973,
    0.6417219919372041, 0.6359117422116274, 0.6546682251619057]
print(mean_std(a))

In [ ]:
# 9,9 - 10E6
(1784.111111111111, 291.42807172208774)
GA & 0.0 (0.0) & 0.627 (0.052) & 464 (16) &  40.6 (1.1) & 653.0 (162.5) & 5349 (301) \\ 
(477.44444444444446, 79.83470577958924)
ABBS & 0.6452 (0.011) & 0.467 (0.103) & 463 (19) &  42.0 (0.2) & 583.9 (186.0) & 1242 (341) \\ 

# 9,9 - 10E5
GA & 0.0 (0.0) & 0.562 (0.045) & 459 (20) &  40.5 (1.1) & 623.2 (178.3) & 5318 (301) \\ 
ABBS & 0.6626 (0.0191) & 0.491 (0.071) & 448 (23) &  41.9 (0.4) & 452.1 (149.7) & 2596 (1517) 

# SFR Optimization

This example seeks to optimize a SFR design utilizing the blackboard framework.
We start first by taking a look at the SFR design data using the `h5` add `reactor` interfaces built into `FRDIGe` [cite].
For this we'll examine the objectives that will drive the optimization process: cycle length, plutonium mass, reactivity swing, and core average burnup.
Upon examination, we note that we have removed a significant portion of the design space by ensuring that the reactor design has a cycle length greater than 0 days.

In [ ]:
h5_interface = h5i.h5Interface()
h5_interface.read_h5('/Users/ryanstewart/projects/sfr_database/SFR_DB_2.h5')

In [ ]:
cycle_length_list = []
rx_swing_list = []
height = []
smear = []
pu_content = []
bu = []
pu_mass = []
print(len(h5_interface.h5file.items()))
for rx, core in h5_interface.h5file.items():
    try:
        rx_ = RI.reactorInterface(core)
        cycle_length = rx_.extrapolate_value('keff', 'time', 1.0)
        rx_swing = rx_.get_reactivity_swing(1.0, cycle_length)
        rx_swing_per_day = rx_swing/cycle_length * 30
        avg_bu = rx_.get_assembly_avg(cycle_length, 'burnup')
        avg_bu_per_day = avg_bu/cycle_length * 100
        pu = rx_.get_assembly_pu_mass() * 78 * 365 / (3 * cycle_length)
        if cycle_length:
            height.append(rx_.rx['independent variables']['height'][0])
            smear.append(rx_.rx['independent variables']['smear'][0])
            pu_content.append(rx_.rx['independent variables']['pu_content'][0])
            cycle_length_list.append(cycle_length)
            rx_swing_list.append(rx_swing_per_day)
            bu.append(avg_bu)
            pu_mass.append(pu)
    except (KeyError, TypeError):
        print('didnt get {}'.format(core))
fig1 = px.scatter_3d(x=height, y=smear, z=pu_content, labels={'x': 'height', 'y': 'smear', 'z': 'pu content'})
fig1.show()


In [ ]:
for s, h, p, pu, c, b, r in zip(smear, height, pu_content, pu_mass, cycle_length_list, bu, rx_swing_list):
    print(s, h, round(p,2), round(pu,0), round(c,0), round(b,0), round(r,0))

# Starting the Blackboard System

In [ ]:
dict_ = {}
for i in range(20):
    ns = run_nameserver()
    bb = run_agent(name='bb', base=bb_opt.BbOpt)
    bb.set_attr(archive_name='mc_2d_1E5/sfr_bb_{}.h5'.format(i))
    bb.load_h5(panels={2:['new','old'],3:['new','old']})
    objectives={'reactivity swing': {'ll':0,   'ul':1500, 'goal':'lt', 'variable type': float},
                        'burnup': {'ll':0,   'ul':100,  'goal':'gt', 'variable type': float}}
    bb.set_attr(objectives=objectives)
    lvl_3 = {}
    for panel in bb.get_attr('abstract_lvls')['level 3'].values():
        lvl_3.update(panel)

    lvl_1 = bb.get_attr('abstract_lvls')['level 1']
 #   print(len(lvl_1))
    print(bb.get_attr('abstract_lvls')['level 100']['final'])
  #  print(len(lvl_3))
   # print(len(lvl_1)/len(lvl_3))
    dict_[i] = (len(lvl_1), len(lvl_3), len(lvl_1)/len(lvl_3))
    ns.shutdown()
    time.sleep(0.5)
print(dict_)

In [ ]:
print(mean_std([x[0] for x in dict_.values()]))
print(mean_std([x[1] for x in dict_.values()]))
print(mean_std([x[2] for x in dict_.values()]))

In [ ]:
ns = run_nameserver()
bb = run_agent(name='bb', base=bb_opt.BbOpt)
bb.set_attr(archive_name='mc_3d_1E5/sfr_bb_0.h5')
bb.load_h5(panels={2:['new','old'],3:['new','old']})
objectives={'reactivity swing': {'ll':0,   'ul':1500, 'goal':'lt', 'variable type': float},
                        'burnup': {'ll':0,   'ul':100,  'goal':'gt', 'variable type': float}}
objectives = {'reactivity swing': {'ll':250,   'ul':1000,  'goal':'lt', 'variable type': float},
              'burnup':           {'ll':30,   'ul':80,  'goal':'gt', 'variable type': float},
              'pu mass':          {'ll':0,   'ul':1000, 'goal':'lt', 'variable type': float}}
bb.set_attr(objectives=objectives)
lvl_3 = {}
for panel in bb.get_attr('abstract_lvls')['level 3'].values():
    lvl_3.update(panel)

lvl_1 = bb.get_attr('abstract_lvls')['level 1']
print(len(lvl_1))
print(bb.get_attr('abstract_lvls')['level 100']['final'])
print(len(lvl_3))
print(len(lvl_1)/len(lvl_3))
time.sleep(0.5)
pu_content = []
height = []
fitness = []
smear = []
obj_dict = {}
for core, values in lvl_1.items():
    fitness.append(round(values['fitness function'],5))
    dv = lvl_3[core]['design variables']
    core_params = lvl_3[core]['objective functions']
    pu_content.append(dv['pu_content'])  
    height.append(dv['height'])
    smear.append(dv['smear'])
    for obj in bb.get_attr('objectives').keys():
        if obj in obj_dict.keys():
            obj_dict[obj].append(core_params[obj])
        else:
            obj_dict[obj] = [core_params[obj]]
objs = [x for x in bb.get_attr('objectives').keys()]
obj_labels = {'fitness': 'fitness',
              'burnup': 'Burnup (GWD/MTHM)',
              'cycle length': 'Cycle Length(days)',
              'pu mass': 'Pu Mass (kg/year)',
              'reactivity swing' : 'Rx Swing (pcm/month)'}

fig1 = px.scatter(x=obj_dict[objs[0]], y=obj_dict[objs[1]],
                  labels={'x':obj_labels[objs[0]], 'y': obj_labels[objs[1]]})
fig1.show()
fig1 = px.scatter(x=obj_dict[objs[1]], y=obj_dict[objs[2]],
                  labels={'x':obj_labels[objs[1]], 'y': obj_labels[objs[2]]})
fig1.show()
fig1 = px.scatter(x=obj_dict[objs[2]], y=obj_dict[objs[0]],
                  labels={'x':obj_labels[objs[2]], 'y': obj_labels[objs[0]]})
fig1.show()
#fig1 = px.scatter(x=obj_dict[objs[0]], y=obj_dict[objs[1]], color=fitness,
#                  labels={'x':obj_labels[objs[0]], 'y': obj_labels[objs[1]], 'color': 'fitness'})
#fig1.show()

#for i in range(0,2):
#    for j in range(0,2):
#        for k in ['height', 'smear', 'pu_content']:
#            if i != j:
#                if k == 'height': colors = height
#                if k == 'smear': colors = smear
#                if k == 'pu_content': colors = pu_content
#                    
#                fig1 = px.scatter(x=obj_dict[objs[i]], y=obj_dict[objs[j]], color=colors,
#                  labels={'x':obj_labels[objs[i]], 'y': obj_labels[objs[j]], 'color': '{}'.format(k)})
#                fig1.show()
#if len(obj_dict) > 2:
#    for k in ['height', 'smear', 'pu_content']:
#        if k == 'height': colors = height
#        if k == 'smear': colors = smear
#        if k == 'pu_content': colors = pu_content
#        fig1 = px.scatter_3d(x=obj_dict[objs[0]], y=obj_dict[objs[1]], z=obj_dict[objs[2]], color=colors, 
#                             labels={'x':obj_labels[objs[0]], 'y': obj_labels[objs[1]], 'z':obj_labels[objs[2]], 'color':k})
#        fig1.show()


fig1 = px.scatter_3d(x=obj_dict[objs[0]], y=obj_dict[objs[1]], z=obj_dict[objs[2]], labels={'x':obj_labels[objs[0]], 'y': obj_labels[objs[1]], 'z':obj_labels[objs[2]]})
fig1.show()

#fig1 = px.scatter_3d(x=height, y=smear, z=pu_content, color=obj_dict[objs[0]], labels={'x':'Height (cm)', 'y': 'Smear', 'z': 'Pu Fraction', 'color':obj_labels[objs[0]]})
#fig1.show()
#fig1 = px.scatter_3d(x=height, y=smear, z=pu_content, color=obj_dict[objs[1]], labels={'x':'Height (cm)', 'y': 'Smear', 'z': 'Pu Fraction', 'color':obj_labels[objs[1]]})
#fig1.show()
#fig1 = px.scatter_3d(x=height, y=smear, z=pu_content, labels={'x':'Height (cm)', 'y': 'Smear', 'z': 'Pu Fraction'})
#fig1.show()
fig1 = px.scatter_3d(x=height, y=smear, z=pu_content, color=fitness, labels={'x':'height', 'y': 'smear', 'z': 'pu content', 'label': 'fitness'})
fig1.show()

### Multi-Agent Test

In [ ]:
import fridge
import fridge.utilities.h5Interface as h5i
import fridge.utilities.reactorInterface as RI
import os
import pickle

from scipy.interpolate import interp1d as interp
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

import osbrain
from osbrain import run_nameserver
from osbrain import run_agent
from osbrain import logging
import numpy as np
import blackboard
import ka
import ka_rp as karp
import ka_br as kabr
import bb_opt
import controller
import time
import performance_measure as pm
import math
import h5py

import pandas as pd
import plotly.graph_objects as go

objectives = {'reactivity swing': {'ll':250,  'ul':1000,  'goal':'lt', 'variable type': float},
              'burnup':           {'ll':30,   'ul':80,    'goal':'gt', 'variable type': float},
              'pu mass':          {'ll':0,    'ul':1000,  'goal':'lt', 'variable type': float}}
#objectives = {'reactivity swing': {'ll':250,   'ul':1000,  'goal':'lt', 'variable type': float},
#              'burnup':           {'ll':30,   'ul':80,    'goal':'gt', 'variable type': float},
#              'pu mass':          {'ll':0,   'ul':1000, 'goal':'lt', 'variable type': float},
#              'cycle length':     {'ll': 120, 'ul': 360, 'goal':'gt', 'variable type': float}}

objectives = {'reactivity swing': {'ll':250,   'ul':1000, 'goal':'lt', 'variable type': float},
              'burnup':           {'ll':30,   'ul':80,  'goal':'gt', 'variable type': float}}
design_variables = {'height':     {'ll': 50.0, 'ul': 100.0, 'variable type': float},
                    'smear':      {'ll': 50.0, 'ul': 80.0, 'variable type': float},
                    'pu_content': {'ll': 0.0,  'ul': 1.0,  'variable type': float}}
kas = {'ka_mc1': karp.KaGlobal,
       'ka_mc2': karp.KaGlobal,
       'ka_mc3': karp.KaGlobal,
       'ka_mc4': karp.KaGlobal,
       'ka_mc': karp.KaGlobal,
       'ka_pert': karp.KaLocal,
       'ka_hc': karp.KaLocalHC,
       'ka_br_lvl3': kabr.KaBr_lvl3,
       'ka_br_lvl2': kabr.KaBr_lvl2,
       'ka_br_lvl1': kabr.KaBr_lvl1}

for x in [0]:#range(5,10):#range(25,26):
    bb_controller = controller.Controller(bb_name='sfr_opt', 
                                          bb_type=bb_opt.BbOpt, 
                                          ka=kas, 
                                          objectives=objectives,
                                          design_variables=design_variables,
                                          archive='sfr_bb_{}'.format(x), 
                                          agent_wait_time=60*120, 
                                          plot_progress=True,
                                          convergence_model={'type':'hvi', 'convegence rate': 1E-4},
                                          progress_rate=20,
                                          surrogate_model={'sm_type': 'gpr', 'pickle file': 'sm_gpr_120days.pkl'})
    bb = bb_controller.bb
    bb.set_attr(num_calls=10)
    bb.set_attr(required_agents=[karp.KaGlobal, karp.KaLocal, karp.KaLocalHC, kabr.KaBr_lvl3, kabr.KaBr_lvl2, kabr.KaBr_lvl1])
    #bb.set_attr(_DEBUG=True)
    ka = bb.get_attr('_proxy_server')
    ka1 = ka.proxy('ka_mc1')
    ka2 = ka.proxy('ka_pert')
    ka3 = ka.proxy('ka_hc')
    br1 = ka.proxy('ka_br_lvl1')
    br1.set_attr(total_pf_size=200)
    br1.set_attr(pareto_sorter='dci')
    br1.set_attr(dci_div={'reactivity swing': 150, 'burnup': 50})
    for ka_ in [ka1, ka2, ka3]:
        ka_.set_attr(_design_accuracy=3)
        ka_.set_attr(_objective_accuracy=3)
    ka2.set_attr(_base_trigger_val=1)
    ka3.set_attr(step_limit=15)
    
    bb_controller.run_multi_agent_bb()
    print(bb.get_attr('abstract_lvls')['level 100']['final'])
    print(len(bb.get_attr('abstract_lvls')['level 1']))
    bb_controller.shutdown()
    time.sleep(2)